In [2]:
import jieba

seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/fh/grdldj5d5h90wkfrhrdxpbp40000gn/T/jieba.cache
Loading model cost 1.265 seconds.
Prefix dict has been built succesfully.


Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


In [9]:
# 用户添加自定词库 jieba.load_userdict(file_name)
'''
词典格式和 dict.txt 一样，一个词占一行；每一行分三部分：
词语、词频（可省略）、词性（可省略），用空格隔开，顺序不可颠倒。
file_name 若为路径或二进制方式打开的文件，则文件必须为 UTF-8 编码。
'''
str_demo = '李小福是创新办主任也是云计算方面的专家'
jieba.load_userdict('user_dict.txt')
seg_user = jieba.cut(str_demo)
print('|'.join(seg_user))

李小福|是|创新办|主任|也|是|云计算|方面|的|专家


In [13]:
# 调整词典
'''
使用 add_word(word, freq=None, tag=None) 和 del_word(word) 可在程序中动态修改词典。
使用 suggest_freq(segment, tune=True) 可调节单个词语的词频，使其能（或不能）被分出来。
注意：自动计算的词频在使用 HMM 新词发现功能时可能无效。
'''

test_str = '如果放到post中将出错。'
seg1 = jieba.cut(test_str, HMM=False)
print('seg1: ' + '|'.join(seg1))
# 改变词频
jieba.suggest_freq(('中', '将'), True)
seg2 = jieba.cut(test_str, HMM=False)
print('seg2: ' + '|'.join(seg2))

test_str2 = '「台中」正确应该不会被切开'
jieba.suggest_freq('台中', True)
print('seg3: ' + '|'.join(jieba.cut(test_str2, HMM=False)))
jieba.suggest_freq(('台', '中'), True)
print('seg4: ' + '|'.join(jieba.cut(test_str2, HMM=False)))

seg1: 如果|放到|post|中|将|出错|。
seg2: 如果|放到|post|中|将|出错|。
seg3: 「|台中|」|正确|应该|不会|被|切开
seg4: 「|台|中|」|正确|应该|不会|被|切开


In [16]:
# 词性标注
import jieba.posseg as pseg

jieba.load_userdict('user_dict.txt')
words = pseg.cut("我爱自然语言处理")
for word, flag in words:
    print('%s %s' % (word, flag))


我 r
爱 v
自然语言处理 x


In [21]:
import sys
import time
sys.path.append("../../")
import jieba

import urllib.request


jieba.enable_parallel()

url = 'http://36kr.com/p/5068410.html'
req = urllib.request.Request(url)
req.add_header('User-Agent', 'Mozilla/5.0')
content = urllib.request.urlopen(req).read()

t1 = time.time()
words = "/ ".join(jieba.cut(content))

t2 = time.time()
tm_cost = t2-t1

log_f = open("1.log","wb")
log_f.write(words.encode('utf-8'))

print('speed %s bytes/second' % (len(content)/tm_cost))

speed 563395.8451429922 bytes/second


In [20]:
# Tokenize：返回词语在原文的起止位置

result = jieba.tokenize(u'永和服装饰品有限公司')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))


word 永和		 start: 0 		 end:2
word 服装		 start: 2 		 end:4
word 饰品		 start: 4 		 end:6
word 有限公司		 start: 6 		 end:10
